In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
import pickle

In [2]:
filepath='/content/airbnb_spreadsheet - AirBnB_NYC.csv'
df=pd.read_csv(filepath)
df.head()

,Host Id,Host Since,Name,Neighbourhood,Property Type,Review Scores Rating (bin),Room Type,Zipcode,Beds,Number of Records,Number Of Reviews,Price,Review Scores Rating
0,500,6/26/2008,Gorgeous 1 BR with Private Balcony,Manhattan,Apartment,NaN,Entire home/apt,10024.0,3.0,1,0,199,NaN
1,500,6/26/2008,Trendy Times Square Loft,Manhattan,Apartment,95.0,Private room,10036.0,3.0,1,39,549,96.0
2,1039,7/25/2008,Big Greenpoint 1BD w/ Skyline View,Brooklyn,Apartment,100.0,Entire home/apt,11222.0,1.0,1,4,149,100.0
3,1783,8/12/2008,Amazing Also,Manhattan,Apartment,100.0,Entire home/apt,10004.0,1.0,1,9,250,100.0
4,2078,8/15/2008,"Colorful, quiet, & near the subway!",Brooklyn,Apartment,90.0,Private room,11201.0,1.0,1,80,90,94.0


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30475 entries, 0 to 30474
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Host Id                     30475 non-null  int64         
 1   Host Since                  30475 non-null  datetime64[ns]
 2   Name                        30475 non-null  object        
 3   Neighbourhood               30475 non-null  object        
 4   Property Type               30472 non-null  object        
 5   Review Scores Rating (bin)  30475 non-null  float64       
 6   Room Type                   30475 non-null  object        
 7   Zipcode                     30341 non-null  float64       
 8   Beds                        30390 non-null  float64       
 9   Number of Records           30475 non-null  int64         
 10  Number Of Reviews           30475 non-null  int64         
 11  Price                       30475 non-null  int64     

In [28]:
df.describe()

,Host Id,Host Since,Review Scores Rating (bin),Zipcode,Beds,Number of Records,Number Of Reviews,Price,Review Scores Rating
count,3.047500e+04,30475,30475.000000,30341.000000,30390.000000,30475.0,30475.000000,30475.000000,22155.000000
mean,1.273040e+07,2013-06-29 17:56:15.317473024,90.536998,10584.830691,1.530141,1.0,12.019852,163.598950,91.993230
min,5.000000e+02,2008-06-26 00:00:00,20.000000,1003.000000,0.000000,1.0,0.000000,10.000000,20.000000
25%,2.700193e+06,2012-06-21 00:00:00,90.000000,10017.000000,1.000000,1.0,0.000000,80.000000,89.000000
50%,8.551260e+06,2013-09-01 00:00:00,90.000000,10065.000000,1.000000,1.0,3.000000,125.000000,94.000000
75%,2.119605e+07,2014-09-12 00:00:00,95.000000,11216.000000,2.000000,1.0,13.000000,195.000000,100.000000
max,4.303307e+07,2015-08-31 00:00:00,100.000000,99135.000000,16.000000,1.0,257.000000,10000.000000,100.000000
std,1.190164e+07,NaN,7.731437,921.324686,1.015395,0.0,21.981496,197.792306,8.850373


In [29]:
df.shape

(30475, 14)

In [4]:
df['Host Since'] = pd.to_datetime(df['Host Since'], errors='coerce')
print(df['Host Since'].head())

0   2008-06-26
1   2008-06-26
2   2008-07-25
3   2008-08-12
4   2008-08-15
Name: Host Since, dtype: datetime64[ns]


Host Tenure- helps to know hw long the host has been active

In [9]:
df['Review Scores Rating (bin)']=pd.to_numeric(df['Review Scores Rating (bin)'],errors='coerce')
ReviewScoresRating_bin_mean=df['Review Scores Rating (bin)'].mean()
df['Review Scores Rating (bin)']=df['Review Scores Rating (bin)'].fillna(ReviewScoresRating_bin_mean)

In [10]:
df.isnull().sum()

,0
Host Id,0
Host Since,0
Name,0
Neighbourhood,0
Property Type,3
Review Scores Rating (bin),0
Room Type,0
Zipcode,134
Beds,85
Number of Records,0


the imputation strategy used here was mean as missing value in review scores can be rightly filled using the average scores.

In [19]:
from sklearn.metrics import mean_squared_error
import numpy as np

# 1. Clean the 'Price' column
df['Price'] = df['Price'].astype(str).str.replace(',', '', regex=False)
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

df.dropna(subset=['Price'], inplace=True)

# 2. Create interaction feature
df['Neighbourhood_RoomType'] = df['Neighbourhood '] + "_" + df['Room Type']

# 3. Prepare features and target
features_no_interaction = ['Neighbourhood ', 'Room Type']
features_with_interaction = ['Neighbourhood ', 'Room Type', 'Neighbourhood_RoomType']

target = 'Price'

# 4. One-hot encode categorical features
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

X_no_interaction = encoder.fit_transform(df[features_no_interaction])
X_with_interaction = encoder.fit_transform(df[features_with_interaction])
y = df[target].values

# 5. Split datasets
X_train_A, X_test_A, y_train_A, y_test_A = train_test_split(X_no_interaction, y, test_size=0.2, random_state=42)
X_train_B, X_test_B, y_train_B, y_test_B = train_test_split(X_with_interaction, y, test_size=0.2, random_state=42)

# 6. Train and evaluate Random Forest on Dataset
rf_B = RandomForestRegressor(random_state=42)
rf_B.fit(X_train_B, y_train_B)
preds_B = rf_B.predict(X_test_B)

mse_B = mean_squared_error(y_test_B, preds_B)
rmse_B = np.sqrt(mse_B)

rf_A = RandomForestRegressor(random_state=42)
rf_A.fit(X_train_A, y_train_A)
preds_A = rf_A.predict(X_test_A)

mse_A = mean_squared_error(y_test_A, preds_A)
rmse_A = np.sqrt(mse_A)

print(f"RMSE without interaction feature: {rmse_A:.2f}")
print(f"RMSE with interaction feature: {rmse_B:.2f}")

RMSE without interaction feature: 169.39
RMSE with interaction feature: 169.39


RMSE without interaction feature: 169.39
RMSE with interaction feature: 169.39
there is no improvement which shows that this interaction doesn’t add predictive value for target (Price)

In [22]:
X = df.drop(columns=['Price', 'Neighbourhood_RoomType'])
y = df['Price']

# 1.Identify categorical and numerical columns
categorical_features = ['Neighbourhood ', 'Property Type', 'Room Type']
# Identify columns to drop
columns_to_drop = ['Host Id', 'Host Since', 'Name', 'Zipcode', 'Beds', 'Number of Records', 'Review Scores Rating']

# Drop identified columns from X
X = X.drop(columns=columns_to_drop)

# Identify remaining numerical columns
numerical_features = X.select_dtypes(include=np.number).columns.tolist()


# Create transformers for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features), # Keep numerical columns as is
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features) # One-hot encode categorical columns
    ],
    remainder='passthrough' # Keep other columns (should be none after dropping)
)

# Create a preprocessing and model pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))])


# 2. Split Data (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Train Random Forest Regressor
pipeline.fit(X_train, y_train)

# 4. Evaluate RMSE
y_pred = pipeline.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')

Root Mean Squared Error (RMSE): 167.6589


RMSE:167.6589 states better accuracy

In [25]:
import joblib

# rf is your trained RandomForestRegressor
joblib.dump(rf, 'random_forest_model.pkl')

['random_forest_model.pkl']